This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/lqr.html).  I recommend having both windows open, side-by-side!


In [ ]:
import matplotlib.pyplot as plt
import mpld3
import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    LogVectorOutput,
    MeshcatVisualizer,
    Parser,
    Simulator,
    StartMeshcat,
)

from underactuated import running_as_notebook

if running_as_notebook:
    mpld3.enable_notebook()

In [ ]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

# LQR on a Manifold

I'll also use these examples to illustrate a little more about how one might build a new model in drake (using multibody plant).  I've included the URDF in this notebook directly as a string so that you can see it and easily play with it.

## Simulating a wheel

*TODO: Consider moving this to the vehicles chapter once it takes more form*

As a warm-up, let's just make sure we understand how to simulate a wheel in Drake (using the full `MultibodyPlant` contact engine).  I've done this by putting the cart on a prismatic joint (so it can't fall down) but now adding collision geometry to the wheel and the ground.

The short story is that if you simply add a wheel (I use a sphere here) and put it into penetration with the ground, then the contact engine will implement the frictional contact which will cause the wheel to roll.  The one subtlety is that the contact is enforced at a point that is a somewhere inside the overlapping region of the wheel and the ground. (We can query it's exact location using the contact results output port of multibodyplant.)  This leads to an effective reduction in the radius of the wheel.

I've put the wheel in this model into relatively deep penetration, just to make the point.

If needed, we could have increased the friction coefficient to make sure that we are in the sticking (not sliding) regime.  But the default parameters worked just fine here.

In [ ]:
cart_with_wheel_urdf = """
<?xml version="1.0"?>

<robot xmlns="http://drake.mit.edu"
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
 name="CartWithWheel">

  <link name="ground">
    <visual>
      <origin xyz="0 0 -5" rpy="0 0 0" />
      <geometry>
        <box size="1000 1000 10" />
      </geometry>
      <material>
        <color rgba="0.93 .74 .4 1" />
      </material>
    </visual>
    <collision>
      <origin xyz="0 0 -5" rpy="0 0 0" />
      <geometry>
        <box size="1000 1000 10" />
      </geometry>
    </collision>
  </link>

  <joint name="ground_weld" type="fixed">
    <parent link="world" />
    <child link="ground" />
  </joint>

  <link name="cart">
    <inertial>
      <mass value="10" />
      <inertia ixx="0" ixy="0" ixz="0" iyy="0" iyz="0" izz="0" />
    </inertial>
    <visual>
      <geometry>
        <box size=".6 .3 .3" />
      </geometry>
      <material>
        <color rgba="0 1 0 1" />
      </material>
    </visual>
  </link>

  <joint name="x" type="prismatic">
    <parent link="world" />
    <child link="cart" />
    <origin xyz="0 0 0.25" />
    <axis xyz="1 0 0" />
  </joint>

  <link name="wheel">
    <inertial>
      <mass value="1" />
      <inertia ixx=".02" ixy="0" ixz="0" iyy="0.02" iyz="0" izz="0.02" />
    </inertial>
    <visual>
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="0 0 0 1" />
      </material>
    </visual>
    <collision>
      <geometry>
        <sphere radius=".05" />
      </geometry>
    </collision>
  </link>

  <joint name="wheel" type="continuous">
    <parent link="cart" />
    <child link="wheel" />
    <!-- I've place the wheel just 0.02 into penetration -->
    <origin xyz="0 0 -.22"/>  
    <axis xyz="0 1 0" />
  </joint>

  <transmission type="SimpleTransmission" name="wheel">
    <actuator name="wheel" />
    <joint name="wheel" />
  </transmission>
  
</robot>
"""

from pydrake.all import PidController


def CartWithWheelExample():
    builder = DiagramBuilder()
    time_step = 0.001

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step)
    Parser(plant).AddModelsFromString(cart_with_wheel_urdf, "urdf")
    plant.Finalize()

    kd_wheel = 1
    pid = builder.AddSystem(
        PidController(
            state_projection=np.array([[0, 1, 0, 0], [0, 0, 0, 1]]),
            kp=[0],
            ki=[0],
            kd=[kd_wheel],
        )
    )
    builder.Connect(pid.get_output_port(), plant.get_actuation_input_port())
    builder.Connect(plant.get_state_output_port(), pid.get_input_port_estimated_state())
    builder.ExportInput(pid.get_input_port_desired_state(), "vd")
    builder.ExportOutput(plant.get_contact_results_output_port(), "contact_results")

    logger = LogVectorOutput(plant.get_state_output_port(), builder)

    # Setup visualization
    meshcat.Delete()
    meshcat.Set2dRenderMode(xmin=-2.5, xmax=2.5, ymin=-1.0, ymax=2.5)
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()

    # Set up a simulator to run this diagram
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    diagram.get_input_port().FixValue(context, [0, 10])

    # Simulate
    visualizer.StartRecording(False)
    simulator.AdvanceTo(2 if running_as_notebook else 0.1)
    visualizer.PublishRecording()
    log = logger.FindLog(context)
    fig, ax = plt.subplots(1, 2, figsize=((10, 6)))
    ax[0].plot(log.sample_times(), log.data().T)
    ax[0].legend(["x", "wheel", "xdot", "wheel_dot"])
    r = 0.05
    print(f"Wheel radius is {r}")
    print(f"x(tf) / wheel(tf) = {log.data()[0,-1]/log.data()[1,-1]}")
    contact_results = diagram.get_output_port().Eval(context)
    z_contact = contact_results.point_pair_contact_info(0).contact_point()[2]
    print(f"The contact point no-slip condition is enforced at z={z_contact}")
    r_effective = r - 0.02 - z_contact
    print(f"This leads to an effective wheel radius of {r_effective}")

    F = np.array([[1, -r_effective, 0, 0], [0, 0, 1, -r_effective]])
    ax[1].plot(log.sample_times(), (F @ log.data()).T)
    ax[1].legend(["x - r_effective*wheel", "xdot - r_effective*wheel_dot"])


CartWithWheelExample()

For a more advanced example, check out the [ballbot](./ballbot.ipynb) notebook next.